<a href="https://colab.research.google.com/github/ndminh2003/Project/blob/main/Play_generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%tensorflow_version 2.x  # this line is not required unless you are in a notebook
from keras.preprocessing import sequence
import keras
import tensorflow as tf
import os
import numpy as np

Colab only includes TensorFlow 2.x; %tensorflow_version has no effect.


In [2]:
path_to_file = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

1130496/1115394 [==============================] - 0s 0us/step


In [3]:
# Read, then decode for py2 compat.
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
# length of text is the number of characters in it
print ('Length of text: {} characters'.format(len(text)))

Length of text: 1115394 characters


In [4]:
# Take a look at the first 250 characters in text
print(text[:250])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.



In [5]:
vocab = sorted(set(text))
# Creating a mapping from unique characters to indices
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

def text_to_int(text):
  return np.array([char2idx[c] for c in text])

text_as_int = text_to_int(text)

In [6]:
# lets look at how part of our text is encoded
print("Text:", text[:13])
print("Encoded:", text_to_int(text[:13]))

Text: First Citizen
Encoded: [18 47 56 57 58  1 15 47 58 47 64 43 52]


In [7]:
def int_to_text(ints):
  try:
    ints = ints.numpy()
  except:
    pass
  return ''.join(idx2char[ints])

print(int_to_text(text_as_int[:13]))

First Citizen


In [8]:
seq_length = 100  # length of sequence for a training example
examples_per_epoch = len(text)//(seq_length+1)

# Create training examples / targets
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

In [9]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

In [10]:
def split_input_target(chunk):  # for the example: hello
    input_text = chunk[:-1]  # hell
    target_text = chunk[1:]  # ello
    return input_text, target_text  # hell, ello

dataset = sequences.map(split_input_target)  # we use map to apply the above function to every entry

In [11]:
for x, y in dataset.take(2):
  print("\n\nEXAMPLE\n")
  print("INPUT")
  print(int_to_text(x))
  print("\nOUTPUT")
  print(int_to_text(y))



EXAMPLE

INPUT
First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You

OUTPUT
irst Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You 


EXAMPLE

INPUT
are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you 

OUTPUT
re all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you k


In [12]:
BATCH_SIZE = 64
VOCAB_SIZE = len(vocab)  # vocab is number of unique characters
EMBEDDING_DIM = 256
RNN_UNITS = 1024

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

data = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

In [13]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim,
                              batch_input_shape=[batch_size, None]),
    tf.keras.layers.LSTM(rnn_units,
                        return_sequences=True,
                        stateful=True,
                        recurrent_initializer='glorot_uniform'),
    tf.keras.layers.Dense(vocab_size)
  ])
  return model

model = build_model(VOCAB_SIZE,EMBEDDING_DIM, RNN_UNITS, BATCH_SIZE)
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (64, None, 256)           16640     
                                                                 
 lstm (LSTM)                 (64, None, 1024)          5246976   
                                                                 
 dense (Dense)               (64, None, 65)            66625     
                                                                 
Total params: 5,330,241
Trainable params: 5,330,241
Non-trainable params: 0
_________________________________________________________________


In [14]:
for input_example_batch, target_example_batch in data.take(1):
  example_batch_predictions = model(input_example_batch)  # ask our model for a prediction on our first batch of training data (64 entries)
  print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")  # print out the output shape

(64, 100, 65) # (batch_size, sequence_length, vocab_size)


In [15]:
# we can see that the predicition is an array of 64 arrays, one for each entry in the batch
print(len(example_batch_predictions))
print(example_batch_predictions)

64
tf.Tensor(
[[[ 3.59176099e-03 -3.93945165e-03 -5.45367086e-03 ... -1.99474016e-04
   -4.66785859e-03 -5.63229714e-03]
  [ 3.01952381e-03 -2.49583181e-03 -5.66403195e-03 ... -7.91824656e-04
    1.65535801e-03 -5.07866545e-03]
  [-9.55192000e-03 -4.84814867e-03 -6.26328820e-03 ... -6.04148896e-04
    3.78282368e-03 -6.31288392e-04]
  ...
  [-6.94018789e-04  5.80838881e-04 -1.64791383e-02 ...  1.18949367e-02
   -8.64649098e-03  8.19950085e-03]
  [-1.81797892e-04  1.96943828e-03 -1.68362930e-02 ...  5.14874235e-03
   -5.27779199e-03  1.11209797e-02]
  [-5.69483964e-03 -1.44291669e-04 -1.17729586e-02 ...  6.44039654e-04
   -4.86962125e-03  4.65851557e-03]]

 [[-5.23264345e-04  2.84367264e-03  1.45340024e-03 ...  1.25428347e-03
    2.26206798e-03  3.78425000e-04]
  [ 3.04812519e-03  9.41042847e-04 -7.42238015e-03 ...  8.28298042e-04
   -3.22986254e-03  5.21536870e-03]
  [ 5.91000263e-03 -6.65985700e-03 -8.23313929e-03 ... -1.03315525e-03
    4.60711727e-03  3.19570932e-03]
  ...
  [ 5.616

In [16]:
# lets examine one prediction
pred = example_batch_predictions[0]
print(len(pred))
print(pred)
# notice this is a 2d array of length 100, where each interior array is the prediction for the next character at each time step

100
tf.Tensor(
[[ 0.00359176 -0.00393945 -0.00545367 ... -0.00019947 -0.00466786
  -0.0056323 ]
 [ 0.00301952 -0.00249583 -0.00566403 ... -0.00079182  0.00165536
  -0.00507867]
 [-0.00955192 -0.00484815 -0.00626329 ... -0.00060415  0.00378282
  -0.00063129]
 ...
 [-0.00069402  0.00058084 -0.01647914 ...  0.01189494 -0.00864649
   0.0081995 ]
 [-0.0001818   0.00196944 -0.01683629 ...  0.00514874 -0.00527779
   0.01112098]
 [-0.00569484 -0.00014429 -0.01177296 ...  0.00064404 -0.00486962
   0.00465852]], shape=(100, 65), dtype=float32)


In [17]:
# and finally well look at a prediction at the first timestep
time_pred = pred[0]
print(len(time_pred))
print(time_pred)
# and of course its 65 values representing the probabillity of each character occuring next

65
tf.Tensor(
[ 3.5917610e-03 -3.9394516e-03 -5.4536709e-03  2.5459931e-03
  1.1389293e-03  9.0083934e-04  1.7996847e-03  3.2723867e-03
 -1.0929516e-03  8.4780436e-03 -6.4213006e-03 -5.8094510e-03
 -6.8529295e-03  4.2464910e-04  6.9220504e-03  2.3133776e-03
 -3.8667661e-03 -2.8500024e-03 -4.2664178e-05 -1.6553054e-04
  2.2476586e-03  1.0900612e-03  3.0927227e-03 -1.6313645e-03
  2.2545638e-03 -2.6969456e-03  3.4609244e-03  2.9292514e-03
  1.8381404e-03  5.8824965e-03  5.7447224e-04 -2.0663161e-04
  4.8485808e-03  1.5003586e-03 -5.6997095e-03 -4.1681868e-03
 -2.3847972e-03  4.1243127e-03  2.1424918e-03  4.3225545e-03
 -3.0105547e-03  1.7251174e-03 -3.5220417e-03  3.0252775e-03
  3.6914444e-03 -2.8800196e-03  2.7425098e-03 -7.8142136e-03
 -5.2357715e-04  1.3000509e-03  8.9242146e-04 -5.9074984e-04
 -4.6104491e-03 -9.0821262e-04  3.7879976e-03  3.3898564e-04
  3.6420645e-03  8.8667357e-04 -4.1858614e-03 -8.2726423e-03
 -1.9657076e-04  1.7001823e-04 -1.9947402e-04 -4.6678586e-03
 -5.632297

In [18]:
# If we want to determine the predicted character we need to sample the output distribution (pick a value based on probabillity)
sampled_indices = tf.random.categorical(pred, num_samples=1)

# now we can reshape that array and convert all the integers to numbers to see the actual characters
sampled_indices = np.reshape(sampled_indices, (1, -1))[0]
predicted_chars = int_to_text(sampled_indices)

predicted_chars  # and this is what the model predicted for training sequence 1

"'LM hWPLNa ?N,k?ZJwwdqQCrwKx '3p.Px'vpCzO;TLOViCVFZ tZ?avtCvbtA:'?US mZjUIDxP.;e&eW\nO3R:eNgAVBOYb,gZ"

In [19]:
def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

In [20]:
model.compile(optimizer='adam', loss=loss)

In [21]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [22]:
history = model.fit(data, epochs=50, callbacks=[checkpoint_callback])

Epoch 1/50
172/172 [==============================] - 14s 64ms/step - loss: 2.5659
Epoch 2/50
172/172 [==============================] - 12s 65ms/step - loss: 1.8646
Epoch 3/50
172/172 [==============================] - 13s 66ms/step - loss: 1.6217
Epoch 4/50
172/172 [==============================] - 13s 67ms/step - loss: 1.4926
Epoch 5/50
172/172 [==============================] - 13s 68ms/step - loss: 1.4139
Epoch 6/50
172/172 [==============================] - 13s 68ms/step - loss: 1.3586
Epoch 7/50
172/172 [==============================] - 13s 70ms/step - loss: 1.3140
Epoch 8/50
172/172 [==============================] - 13s 68ms/step - loss: 1.2751
Epoch 9/50
172/172 [==============================] - 13s 69ms/step - loss: 1.2388
Epoch 10/50
172/172 [==============================] - 13s 68ms/step - loss: 1.2040
Epoch 11/50
172/172 [==============================] - 13s 68ms/step - loss: 1.1680
Epoch 12/50
172/172 [==============================] - 13s 69ms/step - loss: 1.1321
E

In [23]:
model = build_model(VOCAB_SIZE, EMBEDDING_DIM, RNN_UNITS, batch_size=1)

In [24]:
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
model.build(tf.TensorShape([1, None]))

In [26]:
def generate_text(model, start_string):
  # Evaluation step (generating text using the learned model)

  # Number of characters to generate
  num_generate = 800

  # Converting our start string to numbers (vectorizing)
  input_eval = [char2idx[s] for s in start_string]
  input_eval = tf.expand_dims(input_eval, 0)

  # Empty string to store our results
  text_generated = []

  # Low temperatures results in more predictable text.
  # Higher temperatures results in more surprising text.
  # Experiment to find the best setting.
  temperature = 1.0

  # Here batch size == 1
  model.reset_states()
  for i in range(num_generate):
      predictions = model(input_eval)
      # remove the batch dimension
    
      predictions = tf.squeeze(predictions, 0)

      # using a categorical distribution to predict the character returned by the model
      predictions = predictions / temperature
      predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

      # We pass the predicted character as the next input to the model
      # along with the previous hidden state
      input_eval = tf.expand_dims([predicted_id], 0)

      text_generated.append(idx2char[predicted_id])

  return (start_string + ''.join(text_generated))

In [27]:
inp = input("Type a starting string: ")
print(generate_text(model, inp))

Type a starting string: Hello
Hellock, believe your honours
Prove as in armed age to age, my boots
I' the wish despite of all the chests and roots;
His lords have done this feather souls?
You understand me: therefore early then will from
them, my household does!

JOHN OF GAUNT:
Respecting both the unacking my master:
Eng are they must confince them,
As if I weep again:
'Romation as my soldiers: I am
customent with thy life, frighter thus.

HASTINGS:
I tell thee, man, sir.

ABRAHAM:
Do plufer charitation! brothers and Percius was left thee gone,
As thou didst kill of woman ten times for the provost.

HENRY BOLINGBROKE:
Nay Warwicks R KING RICHARD II:
Mine ear is named for approve his gracious stars,
And kneel by Edward cannot well despair,
For ever I war advanced the very witness
Whom we may do it, and would happy from the c
